In [48]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%qtconsole

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"

from IPython.display import display
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
			
import pandas as pd
import numpy as np
from pathlib import Path		

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [2]:
import folium

In [12]:
df = pd.read_csv(r'data/traffic-counts_v1.csv')
df.set_index('Count Start Date', inplace = True)
display_all(df.head())
df.info()

,Road Name,Carriageway Start Name,Carriageway End Name,5 Day ADT,7 Day ADT,Saturday Volume,Sunday Volume,AM Peak Volume,AM Peak Hour,Mid Peak Volume,Mid Peak Hour,PM Peak Volume,PM Peak Hour,Car,LCV,MCV,HCV-I,HCV-II,HCV Total,latitude,longitude
Count Start Date,,,,,,,,,,,,,,,,,,,,,
2015-11-04,LADIES MILE (REMUERA),REMUERA RD,ORMONDE RD,11488.0,10346.0,8471.0,6513.0,1101,745.0,873,1400.0,1212,1700.0,0.95,0.01,0.02,0.01,0.00,0.03,-36.883006,174.809151
2016-05-05,GRANDE VUE RD (MANUREWA),MOTORWAY ONRAMP (END ISLAND LHS),HILL RD,17898.0,17580.0,17770.0,15795.0,1460,NaN,1393,NaN,1279,NaN,0.90,0.01,0.08,0.00,0.00,0.09,-37.014364,174.904840
2017-05-04,GRANDE VUE RD (MANUREWA),MOTORWAY ONRAMP (END ISLAND LHS),HILL RD,18810.0,18014.0,17674.0,14374.0,1572,NaN,1369,NaN,1343,NaN,0.91,0.01,0.07,0.01,0.00,0.08,-37.014364,174.904840
2017-12-14,ST LUKES RD,CORNWALLIS ST,SANDRINGHAM RD (LHS),29674.0,28867.0,28201.0,25499.0,1849,900.0,2355,1130.0,2372,1800.0,0.92,0.01,0.02,0.03,0.01,0.05,-36.884606,174.736356
2017-12-18,KAWAKAWA BAY COAST RD-HC-WK1,TE PAPA RD,END,167.0,200.0,301.0,269.0,18,845.0,41,NaN,39,NaN,0.94,0.02,0.04,0.00,0.00,0.04,-36.938205,175.182248


<class 'pandas.core.frame.DataFrame'>
Index: 2695 entries, 2015-11-04 to 2019-04-09
Data columns (total 21 columns):
Road Name                 2695 non-null object
Carriageway Start Name    2691 non-null object
Carriageway End Name      2694 non-null object
5 Day ADT                 2694 non-null float64
7 Day ADT                 2694 non-null float64
Saturday Volume           2695 non-null float64
Sunday Volume             2695 non-null float64
AM Peak Volume            2695 non-null int64
AM Peak Hour              561 non-null float64
Mid Peak Volume           2695 non-null int64
Mid Peak Hour             867 non-null float64
PM Peak Volume            2695 non-null int64
PM Peak Hour              683 non-null float64
Car                       2695 non-null float64
LCV                       2695 non-null float64
MCV                       2695 non-null float64
HCV-I                     2695 non-null float64
HCV-II                    2695 non-null float64
HCV Total                 2695 

# Map points onto map based on traffic volume

In [15]:
# import plotly.plotly as py
# import plotly.graph_objs as go

In [ ]:
start_year = 2018
start_month = 5
start_day = 1
end_year = 2018
end_month = 6
end_day = 1
start_date = 1

In [24]:
col = '7 Day ADT'

In [51]:
p80 = np.nanpercentile(df[col], 75)
p80

14849.0

In [52]:
df_temp = df.loc['2018-05-01':'2018-05-31']
df_temp = df_temp[df_temp[col] > p80]

In [25]:
import branca
colorscale = branca.colormap.linear.YlGnBu_09.scale(min(df[col]), max(df[col]))

In [53]:
m = folium.Map(location = [-36.848461, 174.763336])  #show Auckland

In [55]:
radius = 5
for long, lat, volume, road in zip(df_temp['longitude'], df_temp['latitude'], df_temp[col], df_temp['Road Name']):
    folium.CircleMarker(location=(lat, long),
                        popup = f'{road}:{volume}',
                        radius=radius,
                        color=colorscale(volume),
                        fill=True).add_to(m)

In [56]:
m